In [10]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
import pandas as pd
from sklearn import cross_validation
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
from sklearn import svm
import math 

In [11]:
train = pd.read_csv("modifiedpima.csv")
train.head()

,6,148,72,35,0,33.6,0.627,50,1
0,8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0,1.0
1,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0,1.0
2,3.0,78.0,50.0,32.0,88.0,31.0,0.248,26.0,1.0
3,2.0,197.0,70.0,45.0,543.0,30.5,0.158,53.0,1.0
4,8.0,125.0,96.0,0.0,0.0,0.0,0.232,54.0,1.0


In [12]:
features = train.columns[0:8]
X = train[features]
y = train['1']
X.head()

,6,148,72,35,0,33.6,0.627,50
0,8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0
1,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0
2,3.0,78.0,50.0,32.0,88.0,31.0,0.248,26.0
3,2.0,197.0,70.0,45.0,543.0,30.5,0.158,53.0
4,8.0,125.0,96.0,0.0,0.0,0.0,0.232,54.0


In [13]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size=0.2,random_state=50)


In [14]:
X_train=np.asarray(X_train)
y_train=np.asarray(y_train)

In [15]:
C=0.1
clf_svm = svm.SVC(kernel='rbf',gamma =0.0001, C=C)
clf_svm.fit(X_train, y_train)
y_pred_svm = clf_svm.predict(X_test) 
acc_svm = accuracy_score(y_test, y_pred_svm)
print ("Overall Linear SVM accuracy: ",acc_svm)

Overall Linear SVM accuracy:  0.7207792207792207


In [40]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=10.0):
    return np.exp((-linalg.norm(x-y)**2) / (2 * (sigma ** 2)))

def gm(y_predict,y_test):
    test_min=0
    test_max=0
    pred_min=0
    pred_max=0
    y_test=np.asarray(y_test)
    for i in range(0,154):
        if(y_test[i]==1):
             test_min=test_min+1
        else:
             test_max=test_max+1
    print("y_test min",test_min)       
    print("y_test max",test_max)
    for i in range(0,154):
        if(y_predict[i]==1):
             pred_min=pred_min+1
        else:
             pred_max=pred_max+1
    print("y_pred min",pred_min)       
    print("y_pred max",pred_max)
    se=pred_min/test_min
    sp=pred_max/test_max
    print(se,sp)
    gm=math.sqrt(se*sp)
    print("GM",gm)

In [41]:
from cvxopt import matrix
class SVM(object):

    def __init__(self, kernel=gaussian_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
    def fit(self, X, y):
        n_samples, n_features = X.shape
        # Gram matrix
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = self.kernel(X[i], X[j])
               # print(K[i,j])
        print(K.shape)

        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
       # print(a)
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        #print(sv)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])
        self.b /= len(self.a)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
                #print(self.w)
        else:
            self.w = None

    def project(self, X):
        if self.w is not None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * self.kernel(X[i], sv)
                y_predict[i] = s
              #  print(y_predict[i])
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [42]:

if __name__ == "__main__":
    import pylab as pl           
    def test_soft():
        
        clf = SVM(C=100.0)
        clf.fit(X_train, y_train)
        y_predict = clf.predict(X_test)
        gm(y_predict,y_test)
        correct = np.sum(y_predict == y_test)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        print("Accuracy",correct/len(y_predict))

    test_soft()    

(613, 613)
     pcost       dcost       gap    pres   dres
 0:  2.6151e+05 -2.1213e+06  2e+06  3e-02  6e-14
 1:  9.9457e+04 -2.4977e+05  4e+05  3e-03  4e-14
 2:  2.1662e+04 -5.9998e+04  8e+04  3e-04  2e-14
 3:  2.6212e+03 -7.2190e+03  1e+04  1e-07  1e-14
 4: -1.5305e+02 -1.3442e+03  1e+03  8e-14  5e-15
 5: -4.1802e+02 -6.6247e+02  2e+02  2e-16  2e-15
 6: -4.5389e+02 -4.9867e+02  4e+01  2e-14  2e-15
 7: -4.5851e+02 -4.6901e+02  1e+01  1e-14  2e-15
 8: -4.5965e+02 -4.6102e+02  1e+00  7e-15  2e-15
 9: -4.5982e+02 -4.5993e+02  1e-01  2e-14  2e-15
10: -4.5984e+02 -4.5984e+02  6e-03  8e-14  1e-15
11: -4.5984e+02 -4.5984e+02  1e-04  5e-14  2e-15
Optimal solution found.
optimal
528 support vectors out of 613 points
y_test min 57
y_test max 97
y_pred min 154
y_pred max 0
2.7017543859649122 0.0
GM 0.0
57 out of 154 predictions correct
Accuracy 0.37012987012987014
